# Задание 1.
Мы будем использовать, встроенный в библиотеку nltk, датасет IMDb Movie Reviews, который можно загрузить с помощью nltk. 

Этот датасет состоит из неструктурированных текстов отзывов, требующих разметки для классификации настроений (положительные/отрицательные).

In [3]:
import nltk
from nltk.corpus import movie_reviews
import random


In [4]:
# Загрузка датасета отзывов
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Николай\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

In [5]:
# Сбор всех отзывов и их меток (положительных/отрицательных)
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [6]:
# Перемешаем датасет
random.shuffle(documents)

In [7]:
# Преобразуем в более удобный формат: текстовый отзыв и его метка
reviews = [" ".join(words) for words, category in documents]
labels = [category for _, category in documents]

In [10]:
# Создадим pandas DataFrame
import pandas as pd
df = pd.DataFrame({'review': reviews, 'label': labels})

# Посмотрим на первые несколько строк
df.head(10)

,review,label
0,"when people are talking about good old times ,...",pos
1,""" practical magic , "" is a film that is so mis...",neg
2,how do films like mouse hunt get into theatres...,neg
3,"if snake eyes were a dog , you ' d put it to s...",neg
4,i had an epiphany today . it occurred to me wh...,neg
5,if you don ' t think kevin kline in drag is fu...,neg
6,"john sayles ' "" men with guns "" is about what ...",pos
7,"you know , i never really wondered what the ta...",neg
8,"directed by : pixote hunt , hendel butoy , eri...",pos
9,"a common complaint amongst film critics is "" w...",pos


## Задание 2: Rule-based разметка
Сначала создадим набор правил для автоматической разметки настроений:


In [11]:
# Правила для классификации на основе ключевых слов
positive_keywords = ["great", "excellent", "amazing", "love", "wonderful", "best"]
negative_keywords = ["bad", "terrible", "boring", "hate", "worst", "awful"]

In [12]:
# Функция для присвоения меток на основе правил
def rule_based_labeling(review):
    review_lower = review.lower()
    if any(word in review_lower for word in positive_keywords):
        return 'pos'
    elif any(word in review_lower for word in negative_keywords):
        return 'neg'
    else:
        return 'neutral'

In [13]:
# Применим rule-based разметку к подмножеству данных
rule_based_df = df.copy()
rule_based_df['rule_label'] = rule_based_df['review'].apply(rule_based_labeling)

# Посмотрим на результат
rule_based_df.head()

,review,label,rule_label
0,"when people are talking about good old times ,...",pos,pos
1,""" practical magic , "" is a film that is so mis...",neg,pos
2,how do films like mouse hunt get into theatres...,neg,pos
3,"if snake eyes were a dog , you ' d put it to s...",neg,pos
4,i had an epiphany today . it occurred to me wh...,neg,neg


# Задача 3: Ручная разметка
Для имитации ручной разметки можно разметить небольшое подмножество данных вручную. Здесь мы просто вручную добавим метки к нескольким отзывам:

In [20]:
# Имитация ручной разметки для подмножества данных
manual_labels = {'I love this movie, it was amazing!': 'pos',
                 'This was a terrible movie, I hated it.': 'neg',
                 'The plot was boring and slow.': 'neg',
                 'It was a wonderful experience watching this.': 'pos'}

In [21]:
# Применим ручные метки к подмножеству данных
manual_df = df.iloc[:4].copy()  # Берем первые 4 отзыва для примера
manual_df['manual_label'] = manual_df['review'].map(manual_labels)

manual_df.head()


,review,label,manual_label
0,"when people are talking about good old times ,...",pos,NaN
1,""" practical magic , "" is a film that is so mis...",neg,NaN
2,how do films like mouse hunt get into theatres...,neg,NaN
3,"if snake eyes were a dog , you ' d put it to s...",neg,NaN


# Задача 4: Объединение данных
Теперь объединим данные с rule-based метками и ручными метками в один набор:

In [31]:
# Объединим вручную размеченные данные с rule-based разметкой
combined_df = pd.concat([manual_df, rule_based_df[~rule_based_df.index.isin(manual_df.index)]], ignore_index=True)

# Проверим, что объединение прошло успешно
combined_df.head()

,review,label,manual_label,rule_label
0,"when people are talking about good old times ,...",pos,NaN,NaN
1,""" practical magic , "" is a film that is so mis...",neg,NaN,NaN
2,how do films like mouse hunt get into theatres...,neg,NaN,NaN
3,"if snake eyes were a dog , you ' d put it to s...",neg,NaN,NaN
4,i had an epiphany today . it occurred to me wh...,neg,NaN,neg


# Задача 5: Обучение модели
Для обучения модели мы преобразуем текстовые данные в числовые признаки с помощью TF-IDF и обучим модель, используя Logistic Regression.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [24]:
# Разделим данные на обучающую и тестовую выборки
X = combined_df['review']
y = combined_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# Преобразуем текст в числовые признаки с использованием TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [26]:
# Обучим модель логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [27]:
# Сделаем предсказания на тестовых данных
y_pred = model.predict(X_test_tfidf)

# Задача 6: Оценка модели
Теперь оценим качество модели на тестовой выборке с помощью метрик:

In [28]:
# Оценим модель
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         neg       0.83      0.82      0.82       201
         pos       0.82      0.83      0.82       199

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

